In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

# **Data Preprocessing**

In [ ]:
# !unzip /kaggle/input/dogs-vs-cats/train.zip
# !unzip /kaggle/input/dogs-vs-cats/test1.zip

In [ ]:
print("train size: ", len(os.listdir('train')))
print("test size: ", len(os.listdir('test1')))

## Visualization

In [ ]:
import matplotlib.pyplot as plt
img = tf.keras.preprocessing.image.load_img('test1/10924.jpg')
plt.imshow(img)

## Splitting data with flow from dataframe

In [ ]:
### category assigning

filenames = os.listdir("train")
categories = []

for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
df.head()

In [ ]:
df["category"] = df["category"].replace({0: 'cat', 1: 'dog'}) 

df.head()

In [ ]:
train_df, valid_df = train_test_split(df, test_size=0.20, random_state=220)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)

valid_df

In [ ]:
BATCH_SIZE = 64

IMAGE_HEIGHT=128
IMAGE_WIDTH=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)


### Train

train_datagen = ImageDataGenerator(rotation_range=15,
                                   rescale=1./255,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1
                                  )


train_generator = train_datagen.flow_from_dataframe(train_df, 
                                                    "train/", 
                                                    x_col='filename',
                                                    y_col='category',
                                                    target_size=IMAGE_SIZE,
                                                    class_mode='binary',
                                                    batch_size=BATCH_SIZE
                                                   )

In [ ]:
## Valid - no augmentation

valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_datagen.flow_from_dataframe(valid_df, 
                                                    "train/", 
                                                    x_col='filename',
                                                    y_col='category',
                                                    target_size=IMAGE_SIZE,
                                                    class_mode='binary',
                                                    batch_size=BATCH_SIZE
                                                   )

## Simple CNN Modeling

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), input_shape=(128, 128, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(32, (3,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.01), 
              loss='binary_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(patience=10)

EPOCHS = 50

history = model.fit(train_generator, 
                    epochs=EPOCHS,
                    validation_data=valid_generator,
                    callbacks=[earlystop]
                   )